In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.src.losses import mean_absolute_error
import tensorflow as tf

In [10]:
from tensorflow.keras.models import load_model

model = load_model('model.keras')

C:\Users\User\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('Housing.csv')
data.replace({'yes': 1, 'no': 0}, inplace=True)
data = data.join(pd.get_dummies(data.furnishingstatus)).drop('furnishingstatus', axis=1)
data.dropna(inplace=True)
x = data.drop(['price'], axis=1)
y = data['price']
xTrain, xTest, yTrain, yTest =  train_test_split(x,y,test_size=0.2)
train_data = xTrain.join(yTrain)
train_data['area'] = np.log(train_data['area']) + 1
train_data['price'] = np.log(train_data['price']) + 1
xTrain, yTrain = train_data.drop(['price'], axis = 1), train_data['price']
test_data = xTest.join(yTest)
test_data['area'] = np.log(test_data['area']) + 1
test_data['price'] = np.log(test_data['price']) + 1
xTest, yTest = test_data.drop(['price'], axis = 1), test_data['price']
selector = SelectKBest(score_func=f_regression, k='all')  #k=8
selector.fit(xTrain, yTrain)
feature_scores = selector.scores_
mask = feature_scores > 20
xTrainReduced = xTrain.loc[:, mask]
xTestReduced = xTest.loc[:, mask]
scaler = StandardScaler()
xTrainScaled = scaler.fit_transform(xTrainReduced)
xTestScaled = scaler.transform(xTestReduced)

C:\Users\User\AppData\Local\Temp\ipykernel_1064\4275164966.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'yes': 1, 'no': 0}, inplace=True)


In [26]:
import math
from random import randint

for i in range(5):
    index = randint(0,len(xTestScaled)-1)
    xSample = xTestScaled[index].reshape(1,-1)
    yTrue = yTest.iloc[index]
    yPred = model.predict(xSample,verbose=0)[0][0]

    truePrice = np.e**(yTrue-1)
    predictedPrice = np.e**(yPred-1)

    print(f"True price for house {index}: {math.ceil(truePrice):,}")
    print(f"Predicted price for house {index}: {predictedPrice:,.2f}\n")

True price for house 7: 8,120,000
Predicted price for house 7: 6,354,057.78

True price for house 96: 3,150,001
Predicted price for house 96: 3,239,413.65

True price for house 4: 5,740,001
Predicted price for house 4: 5,512,336.20

True price for house 25: 5,600,001
Predicted price for house 25: 5,684,052.32

True price for house 55: 5,040,000
Predicted price for house 55: 7,208,679.04

